In [1]:
import imp
import uproot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import ROOT
import time
import math
helper = imp.load_source('helpfunction','/home/tmettler/Desktop/uBoone/do_plots/helpfunction.py')
#import /home/tmettler/Desktop/uBoone/do_plots/helpfunction as helper
from __future__ import division

import collections
NuCC = imp.load_source('NuCC_function','/home/tmettler/Desktop/uBoone/do_plots/NuCC_function.py')


Welcome to JupyROOT 6.10/08


In [2]:
# initialte ROOT default canvas
ROOT.gStyle.SetOptStat(0)
c1 = ROOT.TCanvas("c1","c1",1100,260)
c1.SetGrid(1)
c1.SetLeftMargin(0.14)
c1.SetRightMargin(0.18)
c1.SetBottomMargin(0.14)

ROOT.gStyle.SetOptStat(0)
c2 = ROOT.TCanvas("c2","c2",1100,520)
c2.SetGrid(1)
c2.SetLeftMargin(0.14)
c2.SetRightMargin(0.18)
c2.SetBottomMargin(0.14)
c2.Divide(2,1,0,0)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [3]:
# Load input files

inputdir = '/home/tmettler/Desktop/ub_data/mcc9.1/v08_00_00_33/'
outputdir = inputdir+'NumuCC_MuonCandidate/' 
outputdir_png, outputdir_root = NuCC.prepareOutput(outputdir)

#filename_overlay = 'NuCCana_Overlay_V18_V4.root'
filename_overlay = '10kevents/NuCCana_overlay_all.root'
filename_data = 'NuCCana_ext_2500ev.root'
filename_ext = 'NuCCana_overlay_25ev.root'
filename_dirt = 'NuCCana_dirt_2733ev.root'
    
tree_name = 'numuCCAna'

In [4]:
#Open all the trees of the four files (data, ext, dirt, overlay)

data, ext, dirt, overlay = NuCC.openTrees(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
NuCC.printNumberOfEntries(data,ext,dirt,overlay)

pot_overlay = NuCC.getPOT(inputdir,filename_overlay,tree_name)
pot_dirt =  NuCC.getPOT(inputdir,filename_dirt,tree_name)

pot_data =      2500*4.8e12 # tor860_wcut
data_trigger = 2500.0 #4743794 # 1987072.0 # E1DCNT_wcut
ext_trigger =  2500*600.0  #2120135 #5685315 # EXT
'''
pot_data =     1.25795274297e+18# tor860_wcut
data_trigger = 1359818.0*0.4 # E1DCNT
ext_trigger =  1359818.0  #2120135 #5685315 # EXT
pot_overlay = 1.25795274297e+18/0.5
pot_dirt = 1.25795274297e18/0.04
'''
print 'POT: '
print 'Data:\t\t', pot_data
print 'Ext:\t\t', 0
print 'Overlay:\t', pot_overlay
print 'Dirt:\t\t', pot_dirt
print ''
sample = [data,ext,overlay,dirt]
scale = {data:1.0,ext:1.0,overlay:1.0,dirt:1.0}
name = {data:'data',ext:'ext',overlay:'overlay',dirt:'dirt'}

scale[data], scale[ext], scale[dirt], scale[overlay] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)

scale[dirt] = scale[dirt]*1.3
scale[overlay] = scale[overlay]*1.3
print 'Scalefactors: '
print 'Data:\t\t', scale[data]
print 'Ext:\t\t', scale[ext]
print 'Overlay:\t', scale[overlay]
print 'Dirt:\t\t', scale[dirt]


Overlay: Number of Entries:	140000
Data: Number of Entries:	2500
Ext: Number of Entries:		2500
Dirt: Number of Entries:	2733

POT: 
Data:		1.2e+16
Ext:		0
Overlay:	7.65886530961e+20
Dirt:		8.33883890898e+18

Scalefactors: 
Data:		1.0
Ext:		0.00166666666667
Overlay:	2.03685524805e-05
Dirt:		0.00187076404404


Warning in <TClass::Init>: no dictionary for class ROOT::TIOFeatures is available


In [5]:
if 1:
    filename_overlay = '10kevents/NuCCana_overlay_all.rootout33.root'
    #filename_overlay = 'NuCCana_overlay_points_1kev.rootout2.root'
    filename_data = 'NuCCana_ext_2500ev.rootout33.root'
    filename_ext = 'NuCCana_ext_2500ev.rootout33.root'
    filename_dirt = 'NuCCana_dirt_2733ev.rootout33.root'

    tree_name = 't_out'

    data_out, ext_out, dirt_out, overlay_out = NuCC.openTreesOut(inputdir, filename_data, filename_ext, filename_dirt, filename_overlay, tree_name)
    NuCC.printNumberOfEntries(data_out,ext_out,dirt_out,overlay_out)

    sample_out = [data_out,ext_out,overlay_out,dirt_out]
    scale_out = {data_out:1.0,ext_out:1.0,overlay_out:1.0,dirt_out:1.0}
    name_out = {data_out:'data',ext_out:'ext',overlay_out:'overlay',dirt_out:'dirt'}

    scale_out[data_out], scale_out[ext_out], scale_out[dirt_out], scale_out[overlay_out] = NuCC.calculateScale(data_trigger, ext_trigger, pot_data, pot_dirt, pot_overlay)
    scale_out[dirt_out] = scale_out[dirt_out]*1.3
    scale_out[overlay_out] = scale_out[overlay_out]*1.3

Overlay: Number of Entries:	139999
Data: Number of Entries:	2499
Ext: Number of Entries:		2499
Dirt: Number of Entries:	2732



In [6]:
# Define signals

fidVol = '(Nu_Vx_sce>(-1.55+10) && Nu_Vx_sce<(254.8-10)) && (Nu_Vy_sce>(-115.53+10) && Nu_Vy_sce<(117.47-10)) &&(Nu_Vz_sce>(-0.1+10) && Nu_Vz_sce<(1036.9-50))'
MCfidVol = '(MCNu_Vx>(-1.55+10) && MCNu_Vx<(254.8-10)) && (MCNu_Vy>(-115.53+10) && MCNu_Vy<(117.47-10)) &&(MCNu_Vz>(-0.1+10) && MCNu_Vz<(1036.9-50))'
numu_signal = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG==13 && MCTrackPurity>0.5' # numu CC signal definition
numu_true = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14' # numu CC signal definition
numu_nomu = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPDG!=13 && MCTrackPurity>0.5' # not an MC muon
numu_lowpur = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14 && MCTrackPurity<0.5' #low purity
numu_nc = 'fidVol && MCfidVol && MCNu_CCNC==1' # nutral current
numu_ov = 'fidVol && !MCfidVol' # out of fiducial
numu_other = 'fidVol && MCfidVol && MCNu_CCNC==0 && MCNu_PDG!=14' # e.g anti nu or nue
#signal = 'MCfidVol && MCNu_CCNC==0 && MCNu_PDG==14'
for x in sample:
    # x.SetAlias('muon','(muon_candidate_key==track_key)')
    x.SetAlias('muon','(muon_candidate_key==track_key)')
    
num_fidVol = {}
for x in sample:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)
    num_fidVol[x] = x.GetEntries('fidVol && muon')*scale[x]
    
tot_num_fidVol = num_fidVol[ext]+num_fidVol[dirt]+num_fidVol[overlay]
overlay_signals = {'numu_signal','numu_nomu','numu_lowpur','numu_nc','numu_ov','numu_other'}

In [24]:
for x in sample_out:
    x.SetAlias('muon','(key_muprotratio==track_key)')
    
num_fidVol = {}
for x in sample_out:
    x.SetAlias('fidVol',fidVol)
    x.SetAlias('MCfidVol',MCfidVol)
    x.SetAlias('numu_signal',numu_signal)
    x.SetAlias('numu_true',numu_true)
    x.SetAlias('numu_nomu',numu_nomu)
    x.SetAlias('numu_lowpur',numu_lowpur)
    x.SetAlias('numu_nc',numu_nc)
    x.SetAlias('numu_ov',numu_ov)
    x.SetAlias('numu_other',numu_other)
    #num_fidVol[x] = x.GetEntries('fidVol && muon')*scale[x]
    
#tot_num_fidVol = num_fidVol[ext]+num_fidVol[dirt]+num_fidVol[overlay]
#overlay_signals = {'numu_signal','numu_nomu','numu_lowpur','numu_nc','numu_ov','numu_other'}

In [25]:
# Load the global variables for access of functions

NuCC.loadGlobal(data,ext,dirt,overlay,data_out,ext_out,dirt_out,overlay_out,scale,scale_out,tot_num_fidVol,overlay_signals,sample,sample_out, name,name_out, outputdir_png, outputdir_root)
#NuCC.printGlobal()

In [26]:
track_start_border_x = '(TrackStart_x_sce <(-1.55+10) || TrackStart_x_sce > (254.8-10))'
track_end_border_x = '(TrackEnd_x_sce <(-1.55+10) || TrackEnd_x_sce > (254.8-10))'
track_start_border_y = '(TrackStart_y_sce <(-115.53+10) || TrackStart_y_sce > (117.47-10))'
track_end_border_y = '(TrackEnd_y_sce <(-115.53+10) || TrackEnd_y_sce > (117.47-10))'
track_start_border_z = '(TrackStart_z_sce <(0.1+10) || TrackStart_z_sce > (1036.9-10))'
track_end_border_z = '(TrackEnd_z_sce <(0.1+10) || TrackEnd_z_sce > (1039.9-10))'

track_end_uncontained = '(' + track_end_border_x + ' || ' + track_end_border_y + ' || ' + track_end_border_z+ ')'


data.SetAlias("track_end_uncontained",track_end_uncontained)
ext.SetAlias("track_end_uncontained",track_end_uncontained)
overlay.SetAlias("track_end_uncontained",track_end_uncontained)
dirt.SetAlias("track_end_uncontained",track_end_uncontained)
data_out.SetAlias("track_end_uncontained",track_end_uncontained)
ext_out.SetAlias("track_end_uncontained",track_end_uncontained)
overlay_out.SetAlias("track_end_uncontained",track_end_uncontained)
dirt_out.SetAlias("track_end_uncontained",track_end_uncontained)

data.SetAlias("crt_cut","(abs(crtt0_time-(69000-crt_trig_corr_med)/1000-4)<0.8 || crtt0_time==-1)")
ext.SetAlias("crt_cut","(abs(crtt0_time-(69000-crt_trig_corr_med)/1000-3.57+3.195-4)<0.8 || crtt0_time==-1)")
overlay.SetAlias("crt_cut","(abs(crtt0_time-4)<0.8 || crtt0_time==-1)")
dirt.SetAlias("crt_cut","(abs(crtt0_time-4)<0.8 || crtt0_time==-1)")
data_out.SetAlias("crt_cut","(abs(crtt0_time-(69000-crt_trig_corr_med)/1000-4)<0.8 || crtt0_time==-1)")
ext_out.SetAlias("crt_cut","(abs(crtt0_time-(69000-crt_trig_corr_med)/1000-3.57+3.195-4)<0.8 || crtt0_time==-1)")
overlay_out.SetAlias("crt_cut","(abs(crtt0_time-4)<0.8 || crtt0_time==-1)")
dirt_out.SetAlias("crt_cut","(abs(crtt0_time-4)<0.8 || crtt0_time==-1)")


crt_tom_cut = ' nr_crthit_top==0 && crthit_vertex_zcut==0 && (track_end_uncontained==1 || nr_crthit_beam_tres==0) && crt_cut'
#crt_tom_cut = ' nr_crthit_top==0 && crthit_vertex_zcut==0 && (track_end_uncontained==1 || nr_crthit_beam_tres==0)'
data.SetAlias("crt_tom_cut",crt_tom_cut)
ext.SetAlias("crt_tom_cut",crt_tom_cut)
overlay.SetAlias("crt_tom_cut",crt_tom_cut)
dirt.SetAlias("crt_tom_cut",crt_tom_cut)
data_out.SetAlias("crt_tom_cut",crt_tom_cut)
ext_out.SetAlias("crt_tom_cut",crt_tom_cut)
overlay_out.SetAlias("crt_tom_cut",crt_tom_cut)
dirt_out.SetAlias("crt_tom_cut",crt_tom_cut)

TrackLength_cut = 'TrackLength>8'
data.SetAlias("TrackLength_cut",TrackLength_cut)
ext.SetAlias("TrackLength_cut",TrackLength_cut)
overlay.SetAlias("TrackLength_cut",TrackLength_cut)
dirt.SetAlias("TrackLength_cut",TrackLength_cut)
data_out.SetAlias("TrackLength_cut",TrackLength_cut)
ext_out.SetAlias("TrackLength_cut",TrackLength_cut)
overlay_out.SetAlias("TrackLength_cut",TrackLength_cut)
dirt_out.SetAlias("TrackLength_cut",TrackLength_cut)





True

In [27]:
over = uproot.open(inputdir+filename_overlay)['t_out']

cols = ['NuScore','FlashScore','NumPfp','NuTracks','NumShowers_corr','TrackScore','TrackLength','VtxDistance_sce',\
        'TrackMomRange_mu','TrackMomMCS_mom','TrackPID_chimuon','TrackPID_chiproton',\
        'TrackPID_chipion','ShowerScore','flash_PE','nr_crthit_beam','MCNu_CCNC','MCNu_PDG',\
        'MCNu_Energy','MCNu_LeptonEnergy','MCle_PDG','MCle_Energy','MCle_length','small_wire','wire_plane']

tree_Event = over.pandas.df(cols)
#print([c.decode('UTF-8') for c in over.keys()])

for element in cols:
    print '%20s:\t'%element,tree_Event[element][0]
#print tree_Event['TrackScore'][9]

             NuScore:	0.9999831318855286
          FlashScore:	8.617920875549316
              NumPfp:	5
            NuTracks:	3
     NumShowers_corr:	2
          TrackScore:	0.955219030380249
         TrackLength:	51.061809671330764
     VtxDistance_sce:	0.4589361846446991
    TrackMomRange_mu:	0.22864614
     TrackMomMCS_mom:	0.97899103
    TrackPID_chimuon:	0.0
  TrackPID_chiproton:	0.0
    TrackPID_chipion:	0.0
         ShowerScore:	0.0784958153963089
            flash_PE:	3185.7005286717736
      nr_crthit_beam:	0
           MCNu_CCNC:	0
            MCNu_PDG:	14
         MCNu_Energy:	3.8565795
   MCNu_LeptonEnergy:	-9.0
            MCle_PDG:	-1
         MCle_Energy:	-1.0
         MCle_length:	0.0
          small_wire:	0
          wire_plane:	100


In [28]:
xstart = -500
xend = 500
xbin = 200
#ystart = 0
#yend = 100
#ybin = 100
name = '_PIDproton'
h_1d = ROOT.TH1F("h_1d","Track PID: Proton vs muon",xbin, xstart, xend)#, ybin, ystart, yend)
overlay_out.Draw('TrackPID_chimuon-TrackPID_chiproton>>h_1d','MCle_purity>0.5 && MCle_PDG==13','')
h_1d_p = ROOT.TH1F("h_1d_p","Track PID: Proton vs muon",xbin, xstart, xend)#, ybin, ystart, yend)
overlay_out.Draw('TrackPID_chimuon-TrackPID_chiproton>>h_1d_p','MCle_purity>0.5 && MCle_PDG==2212','')
h_1d.SetXTitle("Chimuon - chi proton")
h_1d.SetYTitle("Number of entries")
h_1d.Draw()
h_1d_p.SetLineColor(2)
h_1d_p.Draw('same')
#c1.SetLogz()
c1.Draw()
c1.SaveAs(outputdir_png + "h_1d"+name+".png")
c1.SaveAs(outputdir_root + "h_1d"+name+".root")

AttributeError: 'PyROOT_NoneType' object has no attribute 'Draw'

Error in <TTreeFormula::Compile>:  Bad numerical expression : "MCle_purity"
Info in <TSelectorDraw::AbortProcess>: Variable compilation failed: {TrackPID_chimuon-TrackPID_chiproton,MCle_purity>0.5 && MCle_PDG==13}
Error in <TTreeFormula::Compile>:  Bad numerical expression : "MCle_purity"
Info in <TSelectorDraw::AbortProcess>: Variable compilation failed: {TrackPID_chimuon-TrackPID_chiproton,MCle_purity>0.5 && MCle_PDG==2212}


In [29]:
NuCC.pdg_content_out('fidVol && muon && TrackLength>8 && crt_tom_cut && (TrackPID_chimuon-TrackPID_chiproton)<-80\
&& (TrackPID_chimuon*100/70-2)<(TrackPID_chipion)')

Total number of entries:  0.0


Warning in <TROOT::Append>: Replacing existing TH1: h_1d (Potential memory leak).


In [30]:
NuCC.pdg_content33("key_longest_track==track_key")

Total number of entries:  0.0


Error in <TTreeFormula::Compile>:  Bad numerical expression : "key_longest_track"
Info in <TSelectorDraw::AbortProcess>: Variable compilation failed: {MCTrackPDG,key_longest_track==track_key}


In [11]:
NuCC.pdg_content33_out("key_longest_track==track_key && num_mc_muon>0 && MCTrackPurity>0.5 && TrackLength>8")

Total number of entries:  38266.0
PDG:  13 	=	93.8% ,	error: 0.5% ,	number:  35885.0
PDG:  2212 	=	4.6% ,	error: 0.1% ,	number:  1775.0
PDG:  211 	=	1.3% ,	error: 0.1% ,	number:  489.0
PDG:  -211 	=	0.2% ,	error: 0.0% ,	number:  78.0
PDG:  -13 	=	0.1% ,	error: 0.0% ,	number:  24.0
PDG:  11 	=	0.0% ,	error: 0.0% ,	number:  6.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  5.0
PDG:  -11 	=	0.0% ,	error: 0.0% ,	number:  4.0


In [12]:
#NuCC.entry33_out('key_longest_track==track_key && num_mc_muon>0 && MCTrackPurity>0.5 && TrackLength>8 && MCTrackPDG==2212',2500)

In [19]:
NuCC.pdg_content33_out("key_muprotdiff==track_key && key_muprotratio==track_key && num_mc_muon>0 && MCTrackPurity>0.5 && TrackLength>8")

Total number of entries:  33327.0
PDG:  13 	=	97.9% ,	error: 0.5% ,	number:  32621.0
PDG:  211 	=	1.3% ,	error: 0.1% ,	number:  422.0
PDG:  2212 	=	0.4% ,	error: 0.0% ,	number:  131.0
PDG:  11 	=	0.2% ,	error: 0.0% ,	number:  64.0
PDG:  -211 	=	0.2% ,	error: 0.0% ,	number:  56.0
PDG:  -11 	=	0.1% ,	error: 0.0% ,	number:  24.0
PDG:  -13 	=	0.0% ,	error: 0.0% ,	number:  8.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  1.0


In [14]:
NuCC.pdg_content33_out("key_muprotratio==track_key && num_mc_muon>0 && MCTrackPurity>0.5 && TrackLength>8")

Total number of entries:  38162.0
PDG:  13 	=	94.0% ,	error: 0.5% ,	number:  35884.0
PDG:  211 	=	2.9% ,	error: 0.1% ,	number:  1107.0
PDG:  2212 	=	2.2% ,	error: 0.1% ,	number:  825.0
PDG:  -211 	=	0.5% ,	error: 0.0% ,	number:  183.0
PDG:  11 	=	0.2% ,	error: 0.0% ,	number:  87.0
PDG:  -11 	=	0.1% ,	error: 0.0% ,	number:  33.0
PDG:  -13 	=	0.1% ,	error: 0.0% ,	number:  33.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  10.0


In [15]:
NuCC.pdg_content33_out("key_mupionratio==track_key && num_mc_muon>0 && MCTrackPurity>0.5 && TrackLength>8")

Total number of entries:  37633.0
PDG:  13 	=	93.7% ,	error: 0.5% ,	number:  35263.0
PDG:  211 	=	3.2% ,	error: 0.1% ,	number:  1192.0
PDG:  2212 	=	1.6% ,	error: 0.1% ,	number:  619.0
PDG:  11 	=	0.7% ,	error: 0.0% ,	number:  247.0
PDG:  -211 	=	0.5% ,	error: 0.0% ,	number:  180.0
PDG:  -11 	=	0.3% ,	error: 0.0% ,	number:  96.0
PDG:  -13 	=	0.1% ,	error: 0.0% ,	number:  29.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  7.0


In [16]:
NuCC.pdg_content33_out("key_muprotratio==track_key && MCTrackPurity>0.5 && TrackLength>8 && crt_tom_cut")

Total number of entries:  43680.0
PDG:  13 	=	77.6% ,	error: 0.4% ,	number:  33892.0
PDG:  2212 	=	13.4% ,	error: 0.2% ,	number:  5835.0
PDG:  211 	=	4.9% ,	error: 0.1% ,	number:  2143.0
PDG:  -211 	=	2.0% ,	error: 0.1% ,	number:  855.0
PDG:  11 	=	0.8% ,	error: 0.0% ,	number:  331.0
PDG:  -13 	=	0.8% ,	error: 0.0% ,	number:  330.0
PDG:  -11 	=	0.6% ,	error: 0.0% ,	number:  271.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  16.0


In [17]:
NuCC.pdg_content33_out("key_muprotdiff==track_key && MCTrackPurity>0.5 && TrackLength>8")

Total number of entries:  45211.0
PDG:  13 	=	78.0% ,	error: 0.4% ,	number:  35270.0
PDG:  2212 	=	12.2% ,	error: 0.2% ,	number:  5519.0
PDG:  211 	=	4.9% ,	error: 0.1% ,	number:  2232.0
PDG:  -211 	=	2.0% ,	error: 0.1% ,	number:  903.0
PDG:  11 	=	1.2% ,	error: 0.1% ,	number:  555.0
PDG:  -11 	=	0.9% ,	error: 0.0% ,	number:  393.0
PDG:  -13 	=	0.7% ,	error: 0.0% ,	number:  326.0
PDG:  321 	=	0.0% ,	error: 0.0% ,	number:  8.0


In [39]:
NuCC.printEffonly_out('muon && TrackLength>8 && crt_tom_cut','overlay_out')

Rejection on each sample:
Keep of overlay_out:	91.26%	 reject:	8.74%
Efficiency:		61.15%
Signal definition=  numu_lowpur	(5.23%)
Signal definition=   numu_other	(0.93%)
Signal definition=    numu_nomu	(7.74%)
Signal definition=      numu_ov	(13.59%)
Signal definition=  numu_signal	(60.63%)
Signal definition=      numu_nc	(11.88%)


In [36]:
NuCC.printEffonly_out('key_muprotratio==track_key && TrackLength>8 && key_muprotdiff==track_key && MCTrackPurity>0.5','overlay_out')

Rejection on each sample:
Keep of overlay_out:	70.97%	 reject:	29.03%
Efficiency:		58.61%
Signal definition=  numu_lowpur	(0.00%)
Signal definition=   numu_other	(0.98%)
Signal definition=    numu_nomu	(5.52%)
Signal definition=      numu_ov	(7.60%)
Signal definition=  numu_signal	(74.72%)
Signal definition=      numu_nc	(11.19%)
